# Variability of dissolved inorganic carbon in the largest karst estuarine lagoon system of the southern Gulf of Mexico

### José Andrés Martínez-Trejo<sup>1</sup>, Jose-Gilberto Cardoso-Mohedano<sup>2,*</sup>, Joan Albert Sanchez-Cabeza<sup>3</sup>, José Martin Hernández Ayón<sup>4</sup>, Ana Carolina Ruiz-Fernández<sup> 3 </sup>, Mario Alejandro Gómez-Ponce <sup>2</sup> , Linda Barranco-Servín<sup>5</sup>, Daniel Pech<sup>5</sup>

<sup> 1 </sup> Posgrado en Ciencias del Mar y Limnología, Universidad Nacional Autónoma de México; Avenida Ciudad Universitaria 3000, C.P. 04510, Coyoacán, Ciudad de México, México

<sup> 2 </sup>  Estación el Carmen, Instituto de Ciencias del Mar y Limnología, Universidad Nacional Autónoma de México, Carretera Carmen-Puerto Real km. 9.5, 24157 Ciudad del Carmen, Campeche, México

<sup> 3 </sup>  Unidad Académica Mazatlán, Instituto de Ciencias del Mar y Limnología, Universidad Nacional Autónoma de México, Calzada Joel Montes Camarena s/n, Colonia Playa Sur, Mazatlán 82000, Sinaloa, México

<sup> 4 </sup>  Instituto de Investigaciones Oceanológicas, Universidad Autónoma de Baja California, Km. 103 CarreteraTijuana-Ensenada, Ensenada, Baja California, México

<sup> 5 </sup> Laboratorio de Biodiversidad Marina y Cambio Climático (BIOMARCCA), El Colegio de la Frontera Sur, Lerma, Campeche, México. 

 \*Corresponding Author: gcardoso@cmarl.unam.mx Phone:+52-938-38-31847

[doi:XXXXX](https://XXXX)


- Raw data are aviable in: 

https://metadata.icmyl.unam.mx/bitstream/handle/20.500.12201/11367/MartinezTrejo_etal_2023_TA_DIC_RawData.csv?sequence=1&isAllowed=y



- Read csv  data file

In [ ]:
# Import packages
import pandas as pd
import PyCO2SYS as pyco2
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats
from scipy.stats import shapiro

In [ ]:
# Function to read in csv file
def read_csv(file):
    """
    Read in csv file and return pandas dataframe
    """
    df = pd.read_csv(file, sep=",", header=0,  decimal=".", encoding='utf-8')
    return df

In [ ]:
# define file path 
terminos_ta_dic_data_path = "../data/MartinezTrejo_etal_2023_TA_DIC_RawData.csv"

# Read in data
terminos_ta_dic_data = read_csv(terminos_ta_dic_data_path)

# Copy data to new dataframe
terminos_ta_dic = terminos_ta_dic_data.copy()

### Calculate DIC / TA ratio and TA- DIC

In [ ]:
# Calculate the ratio of DIC to TA
terminos_ta_dic["DIC/TA"]= terminos_ta_dic["DIC_micromol_kg"]/terminos_ta_dic["TA_micromol_kg"]

# Calculate TA - DIC

terminos_ta_dic["TA-DIC"]= terminos_ta_dic["DIC_micromol_kg"] - terminos_ta_dic["TA_micromol_kg"]


## PyCO2SYS configuration from estuarine waters (Humphreys et al. 2022)

In [ ]:
# Copy data to new dataframe
terminos_inorganic_carbon = terminos_ta_dic.copy()

# Create dictionary with data configuration for PyCO2SYS. 
# For more information on the parameters see Humphreys et al (2022)
my_co2sys_params_dic_ta_config = {
    # DIC measured in the lab in μmol/kg-sw
    "par1": terminos_inorganic_carbon["DIC_micromol_kg"],
    # TA measured in the lab, Total scale
    "par2": terminos_inorganic_carbon["TA_micromol_kg"],
    "par1_type": 2,         # tell PyCO2SYS: "par2 is a DIC value"
    "par2_type": 1,         # tell PyCO2SYS: "par1 is a TA value"
    # Fields conditions
    "salinity": terminos_inorganic_carbon["Sal_psu"],  # in-situ salinity in PSU
    # in-situ temperature (output conditions) in °C
    "temperature_out": terminos_inorganic_carbon["Temp_C"],
    "pressure_out": 0.0,
    # Settings
    "opt_pH_scale": 1,     # Total pH (Wolf-Gladrow et al. 2007)
    "opt_k_carbonic": 15,     # 0 < T < 50 °C, 1 < S < 50, Seawater scale, real seawater (Millero F.J. 2010)
    "temperature": 25      # lab temperature (input conditions) in °C
}

### Run PyCO2SYS

In [ ]:
# Run PyCO2SYS to calculate the carbonate system parameters
terminos_inorganic_carbon_results= pyco2.sys(**my_co2sys_params_dic_ta_config)

### Read the interes variables from PYCO2sys 

In [ ]:
# Select variables of interest from the PyCO2SYS output and create a DataFrame
pyco2_interest_variables = pd.DataFrame({
       "pH": terminos_inorganic_carbon_results["pH"],
       "saturation_aragonite": terminos_inorganic_carbon_results["saturation_aragonite"],
       "pCO2_atm": terminos_inorganic_carbon_results["pCO2_out"],
       "k_aragonite": terminos_inorganic_carbon_results["k_aragonite"],
       "total_calcium": terminos_inorganic_carbon_results["total_calcium"],
       "carbonate": terminos_inorganic_carbon_results["carbonate"],
       "CO3": terminos_inorganic_carbon_results["CO3"]
       
})

# Concatenate PyCO2SYS output with the original dataframe
CarbonateTL = pd.concat([terminos_ta_dic, pyco2_interest_variables], axis=1)

- Terminos Lagoon TA and DIC in TL mean and standard deviation 

In [ ]:
CarbonateTL[['TA_micromol_kg', 'DIC_micromol_kg']].agg(['mean', 'std'])

- Variables by season, mean and standard deviation 

In [ ]:
grouped_data_season = CarbonateTL.groupby("Season")[['TA_micromol_kg','DIC_micromol_kg', 'pCO2_atm', 'pH', 'saturation_aragonite']].agg(['min', 'mean', 'std'])
print(grouped_data_season)

- Select Candelaria and Palizada results

In [ ]:
CandelariaResult = CarbonateTL.loc[CarbonateTL["Estuary"] == "Candelaria"]
                     
PalizadaResult = CarbonateTL.loc[CarbonateTL["Estuary"] == "Palizada"]

- TA vs DIC Perform  least squares (OLS) regression

In [ ]:
#define predictor and response variables
x = CarbonateTL['TA_micromol_kg']
y = CarbonateTL['DIC_micromol_kg']

#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
model = sm.OLS(y, x).fit()



#view model summary
print(model.summary())

# export model summary to text file
with open('../output_files/TA_DIC_liner_model_summary.txt', 'w') as file:
    file.write(model.summary().as_text())

- TA vas DIC linear Equation

In [ ]:

linear_params = model.params

standard_errors = model.bse

Lintercep = f" {linear_params.iloc[0]:.0f}( \pm {standard_errors.iloc[0]:.0f})"

LSlope = f"{linear_params.iloc[1]:.2f}( \pm {standard_errors.iloc[1]:.2f})"

linearEquation= f"$y = {LSlope} \cdot x{Lintercep}$"


print("linear Equation" + linearEquation)

Rsquared = f"$r^{2}$ = {model.rsquared:.3f}"

- TA vs DIC , scatter plot  with linear regretion

In [ ]:

x = CarbonateTL['TA_micromol_kg']
y = CarbonateTL['DIC_micromol_kg']

# Plot 
g = sns.scatterplot(
    data=CarbonateTL,
    x="TA_micromol_kg", y="DIC_micromol_kg", hue="Season",
    palette=dict(Dry="#21918c", Rainy="#440154")
)

# Add labels and title
plt.xlabel('TA ($\mu mol  \; kg^{-1}$)', fontsize = 12)

plt.ylabel('DIC ($\mu mol  \; kg^{-1}$)', fontsize = 12)

# Calculate the linear regression line
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

# Plot linear regression 

plt.plot(x, intercept + slope*x, '#fde725')

# set the figure size
plt.gcf().set_size_inches(6, 4)



plt.text(3300, 2500, linearEquation)

plt.text(3300, 2300, Rsquared)


# save the plot as a PDF file paper format  
plt.savefig('../output_files/DIC_TA.pdf', dpi=300, bbox_inches='tight')

plt.show()


-Salinity  vs DIC , scatter plot  with linear regretion

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats

x = CarbonateTL['Sal_psu']
y = CarbonateTL['DIC_micromol_kg']
import seaborn as sns



# Plot sepal width as a function of sepal_length across days
g = sns.scatterplot(
    data=CarbonateTL,
    x="Sal_psu", y="DIC_micromol_kg", hue="Season",
    palette=dict(Dry="#21918c", Rainy="#440154")
)


#plt.scatter(x, y, label='original data', color = "#3b528b")

# Add labels and title
plt.xlabel('Salinity', fontsize = 12)

plt.ylabel('DIC ($\mu mol  \; kg^{-1}$)', fontsize = 12)

# Calculate the linear regression line
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

# Plot linear regression 

plt.plot(x, intercept + slope*x, '#fde725')

# set the figure size
plt.gcf().set_size_inches(6, 4)

# save the plot as a PDF file paper format 
plt.savefig('../output_files/SAL_DIC.tiff', dpi=300, bbox_inches='tight')

plt.show()

# Print linear regression results 

print("r-squared:", r_value**2)
print("Intercept:", intercept)
print("Slope:", slope)

-Salinity vs DIC ,  Perform  least squares (OLS) regression

In [ ]:
import statsmodels.api as sm

#define predictor and response variables
x = CarbonateTL['Sal_psu']
y = CarbonateTL['DIC_micromol_kg']

#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
model = sm.OLS(y, x).fit()

#view model summary
print(model.summary())

# export model summary to text file
with open('../output_files/Sal_DIC_liner_model_summary.txt', 'w') as file:
    file.write(model.summary().as_text())

-Salinity  vs TA , scatter plot  with linear regretion

In [ ]:
import matplotlib.pyplot as plt
from scipy import stats

x = CarbonateTL['Sal_psu']
y = CarbonateTL['TA_micromol_kg']
import seaborn as sns



# Plot sepal width as a function of sepal_length across days
g = sns.scatterplot(
    data=CarbonateTL,
    x="Sal_psu", y="TA_micromol_kg", hue="Season",
    palette=dict(Dry="#21918c", Rainy="#440154")
)


#plt.scatter(x, y, label='original data', color = "#3b528b")

# Add labels and title
plt.xlabel('Salinity', fontsize = 12)

plt.ylabel('TA ($\mu mol  \; kg^{-1}$)', fontsize = 12)

# Calculate the linear regression line
slope, intercept, r_value, p_value, std_err = stats.linregress(x, y)

# Plot linear regression 

plt.plot(x, intercept + slope*x, '#fde725')

# set the figure size
plt.gcf().set_size_inches(6, 4)

# save the plot as a PDF file paper format 
plt.savefig('../output_files/SAL_TA.tiff', dpi=300, bbox_inches='tight')

plt.show()

# Print linear regression results 

print("r-squared:", r_value**2)
print("Intercept:", intercept)
print("Slope:", slope)

-Salinity vs TA ,  Perform  least squares (OLS) regression

In [ ]:
import statsmodels.api as sm

#define predictor and response variables
x = CarbonateTL['Sal_psu']
y = CarbonateTL['TA_micromol_kg']

#add constant to predictor variables
x = sm.add_constant(x)

#fit linear regression model
model = sm.OLS(y, x).fit()

#view model summary
print(model.summary())

# export model summary to text file
with open('../output_files/Sal_TA_liner_model_summary.txt', 'w') as file:
    file.write(model.summary().as_text())

Plot DIC/TA ratio,  Salinity and pH by Estuary 

In [ ]:
# Create the scatter plot 

# getting the original colormap using cm.get_cmap() function
orig_map = plt.colormaps.get_cmap('viridis')

# reversing the original colormap using reversed() function
reversed_map = orig_map.reversed()

scatter = plt.scatter(
    x=CandelariaResult["Sal_psu"], y=CandelariaResult["DIC_micromol_kg"]/CandelariaResult["TA_micromol_kg"], c=CandelariaResult["pH"], cmap=reversed_map,
    marker='.', s= 300, label='Candelaria', alpha=0.9)


scatter = plt.scatter(
    x=PalizadaResult["Sal_psu"], y=PalizadaResult["DIC_micromol_kg"] / PalizadaResult["TA_micromol_kg"], c=PalizadaResult["pH"], cmap=reversed_map,
    marker='+', s= 300, label='Palizada', alpha=0.9)


# Add a color bar
plt.colorbar(scatter)


# Add labels and title
plt.xlabel('Salinity', fontsize = 12)
plt.ylabel("$DIC \, / \, TA $" , fontsize = 12)
plt.legend(loc='upper right')

# set the figure size
plt.gcf().set_size_inches(6, 4)

# save the plot as a PDF file
plt.savefig('../output_files/sal_DICTA_pH.pdf', dpi=300, bbox_inches='tight')

# display the plot
plt.show()



In [ ]:
# Create the scatter plot 

# getting the original colormap using cm.get_cmap() function
orig_map = plt.colormaps.get_cmap('viridis')

# reversing the original colormap using reversed() function
reversed_map = orig_map.reversed()

scatter = plt.scatter(
    x=CandelariaResult["TA-DIC"], y=CandelariaResult["pH"], c=CandelariaResult["Sal_psu"], cmap=reversed_map,
    marker='.', s= 300, label='Candelaria', alpha=0.9)


scatter = plt.scatter(
    x=PalizadaResult["TA-DIC"], y=PalizadaResult["pH"], c=PalizadaResult["Sal_psu"], cmap=reversed_map,
    marker='+', s= 300, label='Palizada', alpha=0.9)


# Add a color bar
plt.colorbar(scatter)


# Add labels and title
plt.xlabel('[TA-DIC]', fontsize = 12)
plt.ylabel("pH" , fontsize = 12)
plt.legend(loc='upper right')

# set the figure size
plt.gcf().set_size_inches(6, 4)

# save the plot as a PDF file
plt.savefig('../output_files/sal_DICTA_pH.pdf', dpi=300, bbox_inches='tight')

# display the plot
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Create a grid of DIC and TA values


DIC_grid, TA_grid = np.meshgrid(CarbonateTL["DIC_micromol_kg"], CarbonateTL["TA_micromol_kg"])

# Calculate the difference between TA and DIC, just as a simple example
# This is where the actual model to calculate (TA - DIC) would go
# For demonstration, we'll use a simple linear relationship,
# which will not exactly replicate the complex processes shown in the actual diagram
TA_minus_DIC = TA_grid - DIC_grid

# Create contour levels for plotting


contour_levels = np.linspace(-1800, 1800, 8)

# Set up the plot
fig, ax = plt.subplots()

# Create the contour plot
contour = ax.contourf(DIC_grid, TA_grid, TA_minus_DIC, levels=contour_levels, cmap='viridis')

# Add lines and annotations corresponding to processes
# These are arbitrary lines for demonstration purposes
#ax.annotate('Photosynthesis', xy=(1950, 2300), xytext=(1900, 2350),
#            arrowprops=dict(facecolor='black', shrink=0.05))
#ax.annotate('Respiration', xy=(2050, 2300), xytext=(2100, 2350),
#            arrowprops=dict(facecolor='black', shrink=0.05))
#ax.annotate('CaCO3 formation', xy=(1950, 2200), xytext=(1900, 2150),
#            arrowprops=dict(facecolor='black', shrink=0.05))
#ax.annotate('CaCO3 dissolution', xy=(2050, 2200), xytext=(2100, 2150),
#            arrowprops=dict(facecolor='black', shrink=0.05))

# Add labels to axes
ax.set_xlabel('DIC (µmol kg^-1)')
ax.set_ylabel('TA (µmol kg^-1)')

# Add a colorbar
cbar = plt.colorbar(contour)
cbar.set_label('(TA - DIC) (µmol kg^-1)')

# Show the plot
plt.show()


# References
- Humphreys, M. P., Lewis, E. R., Sharp, J. D., & Pierrot, D. (2022). PyCO2SYS v1.8: Marine carbonate system calculations in Python. Geoscientific Model Development, 15(1), 15–43. https://doi.org/10.5194/gmd-15-15-2022
- Millero, F. J. (2010). Carbonate constants for estuarine waters. Marine and Freshwater Research, 61(2), 139. https://doi.org/10.1071/MF09254
- Wolf-Gladrow, D. A., Zeebe, R. E., Klaas, C., Körtzinger, A., & Dickson, A. G. (2007). Total alkalinity: The explicit conservative expression and its application to biogeochemical processes. Marine Chemistry, 106(1–2), 287–300. https://doi.org/10.1016/j.marchem.2007.01.006

## Apparent Oxygen Utilization 

In [ ]:
def convert_oxygen_concentration(oxygen_in_mg_per_L, salinity, temperature, pressure):
    """
    Convert oxygen concentration from mg/L to micromol/kg in seawater.

    Parameters:
    oxygen_in_mg_per_L (float): Oxygen concentration in mg/L
    salinity (float): Absolute Salinity in g/kg
    temperature (float): Conservative Temperature in degrees Celsius
    pressure (float): Sea pressure in dbar

    Returns:
    float: Oxygen concentration in micromol/kg
    """
    import gsw

    # Molecular weight of O2
    molecular_weight_O2 = 31.9988  # g/mol

    # Convert mg/L to mol/L
    oxygen_in_mol_per_L = oxygen_in_mg_per_L / (1000 * molecular_weight_O2)

    # Convert mol/L to micromol/L
    oxygen_in_micromol_per_L = oxygen_in_mol_per_L * 1e6

    # Calculate density of seawater in kg/L
    density_seawater = gsw.rho(salinity, temperature, pressure) / 1000  # kg/L

    # Convert micromol/L to micromol/kg
    oxygen_in_micromol_per_kg = oxygen_in_micromol_per_L / density_seawater

    return oxygen_in_micromol_per_kg

In [ ]:
CarbonateTL["DO_micromol_kg"] = convert_oxygen_concentration(CarbonateTL["DO_mg_L"], CarbonateTL["Sal_psu"], CarbonateTL["Temp_C"], 0.0)

In [ ]:
def calculate_aou(salinity, temperature, pressure, oxygen, latitude, longitude):
    """
    Calculate Apparent Oxygen Utilization (AOU).

    Parameters:
    salinity (float): Absolute Salinity in g/kg; 
    Practical Salinity Unit (PSU) is a unit based on the properties of seawater conductivity. 
    It's a dimensionless unit, but it's roughly equivalent to or grams per kilogram (g/kg)
    temperature (float): Conservative Temperature in degrees Celsius
    pressure (float): Sea pressure in dbar
    oxygen (float): Dissolved oxygen concentration in micromol/kg

    Returns:
    float: AOU in micromol/kg
    Reference:

    TEOS-10 Manual
    GSW-Python Documentation
    """
    import gsw

    # Calculate saturation concentration of oxygen in micromol/kg
    oxygen_sat = gsw.O2sol(salinity, temperature, pressure, latitude, longitude)

    # Calculate AOU
    aou = oxygen_sat - oxygen

    return aou


In [ ]:
CarbonateTL["oxygen_utilization_micromol_kg"] =calculate_aou(CarbonateTL["Sal_psu"], CarbonateTL["Temp_C"],  0.0, CarbonateTL["DO_micromol_kg"], CarbonateTL["latitude"] , CarbonateTL["latitude"] )



In [ ]:
plt.plot(CarbonateTL["oxygen_utilization_micromol_kg"], CarbonateTL["TA_micromol_kg"]-CarbonateTL["DIC_micromol_kg"], 'o', color='blue')

In [ ]:
plt.plot(CarbonateTL["TA_micromol_kg"]- CarbonateTL["DIC_micromol_kg"], CarbonateTL["pH"]  , 'o', color='blue')

# x="TA_micromol_kg", y="DIC_micromol_kg", hue="Season",

In [ ]:
sns.set_theme()


g = sns.lmplot(data=CarbonateTL, x="TA-DIC", y="pH", hue="Estuary", palette=dict(Dry="#21918c", Rainy="#440154"))

In [ ]:
CarbonateTL["pCO2_atm"].describe()